#  Usage/Benchmark Tests
-------------------


In [1]:
# Load Params 
using BenchmarkTools

push!(LOAD_PATH, "$(pwd())/src"); using DarkWingDuck;
const dwd = DarkWingDuck;

In [2]:
"""Generate uniform random points along a surface"""
function genRandomCoords(x0::Float64, x1::Float64, y0::Float64, y1::Float64, n::Int)::Array{dwd.Coord}
    X = ((ones(n) * x0) + (rand(n) * (x1 - x0))) 
    Y = ((ones(n) * y0) + (rand(n) * (y1 - y0)))
    return [dwd.Coord(x, y) for (x,y) in zip(X,Y)]
end

genRandomCoords

In [3]:
const QT_TEST_SIZE, RM_TEST_SIZE = 500_000, 100_000

## Inserts - Tree Build
ps = genRandomCoords(0., 1., 0., 1., QT_TEST_SIZE);

#TODO: constructor should take anyFloat
sampleQTree = dwd.qtBox(dwd.Coord(.0, .0), 1.)

@time for p in ps 
    dwd.insertIntoQuadTree!(sampleQTree, p)
end

  1.078553 seconds (11.44 M allocations: 369.082 MiB, 22.18% gc time)


In [4]:
println("Total Obj Size: $(Base.summarysize(sampleQTree) / 1000000 ) MB")

Total Obj Size: 17.997784 MB


In [5]:
@benchmark dwd.queryRange(sampleQTree, dwd.Box(dwd.Coord(rand() , rand()), rand()/2))

BenchmarkTools.Trial: 
  memory estimate:  8.22 KiB
  allocs estimate:  164
  --------------
  minimum time:     9.026 μs (0.00% GC)
  median time:      2.821 ms (0.00% GC)
  mean time:        5.131 ms (15.30% GC)
  maximum time:     27.340 ms (15.59% GC)
  --------------
  samples:          971
  evals/sample:     1

In [6]:
#Test in kilometers, 5km ~0.045 on a flat plane
@benchmark dwd.radialSearch(sampleQTree, dwd.Coord(rand(), rand()), Float32(5.))

BenchmarkTools.Trial: 
  memory estimate:  161.68 MiB
  allocs estimate:  473212
  --------------
  minimum time:     114.705 ms (8.44% GC)
  median time:      119.611 ms (11.61% GC)
  mean time:        134.521 ms (21.28% GC)
  maximum time:     274.656 ms (59.26% GC)
  --------------
  samples:          38
  evals/sample:     1

In [7]:
# Oyy...function modifies the structure, use @time instead of benchmark
@time (for i in 1:RM_TEST_SIZE dwd.removePoint!(sampleQTree, ps[rand(1:QT_TEST_SIZE)]) end)

  0.640509 seconds (5.81 M allocations: 296.949 MiB, 4.60% gc time)


In [8]:
println("Total Obj Size: $(Base.summarysize(sampleQTree) / 1000000 ) MB")
println("Total Point Count: $(length(dwd.queryRange(sampleQTree, dwd.Box(dwd.Coord(0. , 0.), 1))))")

Total Obj Size: 16.549256 MB
Total Point Count: 409467


In [9]:
@time dwd.cleanUpQTree(sampleQTree)

  0.051978 seconds (166.94 k allocations: 6.479 MiB)


true

In [10]:
println("Total Obj Size: $(Base.summarysize(sampleQTree) / 1000000 ) MB")
println("Total Point Count: $(length(dwd.queryRange(sampleQTree, dwd.Box(dwd.Coord(0. , 0.), 1))))")

Total Obj Size: 16.433832 MB
Total Point Count: 409467
